In [1]:
from sklearn import model_selection, preprocessing, linear_model,naive_bayes,metrics ,svm
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd 
#import xgboost 
import numpy as np 
#import textblob 
import  string 
import keras
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.



# create a dataframe using texts and lables


In [2]:
Df2=pd.read_csv('train_text_data_sim_label2.csv')

In [3]:
Df2.head()

,id,label,tweet
0,1,label_5,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,label_3,Finally a transparant silicon case ^^ Thanks t...
2,3,label_4,We love this! Would you go? #talk #makememorie...
3,4,label_4,I'm wired I know I'm George I was made that wa...
4,5,label_1,What amazing service! Apple won't even talk to...


In [4]:
trainDF=Df2

In [5]:
trainDF['tweet'].head(5)

0    #fingerprint #Pregnancy Test https://goo.gl/h1...
1    Finally a transparant silicon case ^^ Thanks t...
2    We love this! Would you go? #talk #makememorie...
3    I'm wired I know I'm George I was made that wa...
4    What amazing service! Apple won't even talk to...
Name: tweet, dtype: object

In [6]:
trainDF['label'].head(5)

0    label_5
1    label_3
2    label_4
3    label_4
4    label_1
Name: label, dtype: object

In [7]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['tweet'], trainDF['label'])

In [8]:
labels1=trainDF['label']


In [9]:
labels1.head(10)

0    label_5
1    label_3
2    label_4
3    label_4
4    label_1
5    label_1
6    label_5
7    label_4
8    label_4
9    label_3
Name: label, dtype: object

# label encode the target variable 


In [10]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()

In [11]:
lb.fit(trainDF['label'])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [12]:
labels2=lb.transform(trainDF['label'])

In [13]:
labels2

array([[0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1]])

In [14]:
lb.classes_

array(['label_1', 'label_2', 'label_3', 'label_4', 'label_5'], dtype='<U7')

In [15]:
#for train and test 

In [16]:
train_y_Binarizer=lb.transform(train_y)
valid_y_Binarizer=lb.transform(valid_y)

In [17]:
train_y_Binarizer

array([[1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0]])

### encode the text

In [18]:
# integer encode the documents
vocab_size = 70
encoded_docs = [text.one_hot(d, vocab_size) for d in trainDF['tweet']]
print(encoded_docs)

[[45, 57, 7, 47, 40, 30, 11, 55, 51, 13, 64, 34, 22, 30, 4, 49], [51, 64, 11, 11, 43, 48, 68, 12, 69, 21, 16, 36, 17, 10, 55, 27, 6, 53, 2], [27, 19, 60, 34, 22, 34, 67, 50, 62, 30, 49, 12, 69, 69, 55, 62, 65, 41], [66, 54, 22, 2, 66, 42, 22, 6, 10, 39, 54, 49, 64, 53, 4, 55, 32, 3, 53, 57, 2, 52], [23, 49, 17, 54, 27, 63, 67, 68, 65, 6, 64, 51, 22, 44, 4, 22, 45, 2, 69, 7, 45, 45, 54, 47], [49, 36, 40, 40, 59, 12, 53, 68, 33, 54, 60], [60, 45, 13, 5, 19, 13, 16, 36, 57, 47, 27, 6, 53, 59], [42, 2, 42, 3, 54, 5, 55, 35, 56, 19, 5, 58, 45, 62, 11, 57, 20, 42, 28, 21, 15, 68, 27, 3, 32, 49, 48, 68, 26, 62, 68, 48, 29, 48, 57, 67, 66, 6, 28], [9, 68, 34, 49, 12, 11, 68, 21, 49, 30, 21, 28, 38, 55, 32, 3, 53, 1], [6, 21, 45, 14, 16, 16, 62, 27, 58, 60, 49, 64, 13, 55, 62, 19, 69, 1], [30, 54, 10, 22, 4, 64, 42, 4, 15, 4, 27, 64, 42, 65, 22, 69, 3, 4, 27, 15, 57, 27, 47, 29, 5], [47, 15, 43, 27, 7, 27, 6, 23, 22, 53, 27, 68, 54, 15, 40, 58, 16, 29, 39, 53, 54], [51, 21, 32, 68, 58, 49, 8, 5

The sequences have different lengths and Keras prefers inputs to be vectorized and all inputs to have the same length. We will pad all input sequences to have the length of 4. Again, we can do this with a built in Keras function, in this case the pad_sequences() function.

In [19]:
# pad documents to a max length of 4 words
max_length = 70
padded_docs =sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[45 57  7 ...  0  0  0]
 [51 64 11 ...  0  0  0]
 [27 19 60 ...  0  0  0]
 ...
 [36 12 18 ...  0  0  0]
 [51 52 12 ...  0  0  0]
 [54 48 54 ...  0  0  0]]


In [20]:
padded_docs1=padded_docs

In [21]:
# for train and test

In [22]:
encoded_docs_train_x = [text.one_hot(d, vocab_size) for d in train_x]
encoded_docs_valid_x = [text.one_hot(d, vocab_size) for d in valid_x]

In [23]:
encoded_docs_train_x=sequence.pad_sequences(encoded_docs_train_x, maxlen=max_length, padding='post')
encoded_docs_valid_x=sequence.pad_sequences(encoded_docs_valid_x, maxlen=max_length, padding='post')

We are now ready to define our Embedding layer as part of our neural network model.

The Embedding has a vocabulary of 50 and an input length of 4. We will choose a small embedding space of 8 dimensions.

The model is a simple binary classification model. Importantly, the output from the Embedding layer will be 4 vectors of 8 dimensions each, one for each word. We flatten this to a one 32-element vector to pass on to the Dense output layer.

In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import SpatialDropout1D
from keras.layers import Convolution1D
from keras.layers import GlobalMaxPool1D
from keras.layers import Dropout
from keras.layers.embeddings import Embedding

In [37]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
#model.add(SpatialDropout1D(0.3))

# Add the convolutional Layer
model.add(Convolution1D(100, 3, activation="relu"))
# Add the pooling Layer
model.add(GlobalMaxPool1D())
#model.add(SpatialDropout1D(0.3))
# Add the output Layers
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(5, activation="softmax"))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 70, 8)             560       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 68, 100)           2500      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 255       
Total params: 8,365
Trainab

model=create_cnn()
print(model.summary())

In [38]:
# fit the model
model.fit(encoded_docs_train_x, train_y_Binarizer, epochs=50, verbose=0)
# evaluate the model


In [39]:
loss, accuracy = model.evaluate(encoded_docs_valid_x, valid_y_Binarizer, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 78.424242


In [40]:
model.predict(encoded_docs_valid_x)

array([[8.76829267e-01, 1.22820266e-01, 7.44851714e-05, 2.55106745e-04,
        2.08359743e-05],
       [1.26929343e-01, 2.87815809e-01, 1.39392853e-01, 1.76151767e-01,
        2.69710213e-01],
       [2.92331398e-01, 3.13345850e-01, 7.96021000e-02, 1.80773035e-01,
        1.33947626e-01],
       ...,
       [2.63219210e-03, 2.70939479e-03, 2.83705860e-01, 3.21093112e-01,
        3.89859408e-01],
       [8.79196465e-01, 8.36386606e-02, 3.65096331e-03, 5.79994358e-03,
        2.77138501e-02],
       [4.17455303e-04, 9.40982427e-04, 3.30303788e-01, 2.20151246e-01,
        4.48186576e-01]], dtype=float32)

In [41]:
y_proba =np.argmax(model.predict(encoded_docs_train_x))

In [42]:
from keras.utils.np_utils import to_categorical

In [43]:
name_id_map = dict(zip(lb.classes_, range(len(lb.classes_))))

In [44]:
name_id_map

{'label_1': 0, 'label_2': 1, 'label_3': 2, 'label_4': 3, 'label_5': 4}

In [45]:
key_list = list(name_id_map.keys()) 
val_list = list(name_id_map.values())

In [46]:
prob=model.predict(encoded_docs_valid_x)

lables_pred=np.argmax(prob,axis=-1)
print(lables_pred)

[0 1 1 ... 4 0 4]


In [49]:
pridectedList=[]
for i in lables_pred:
    print(key_list[val_list.index(i)])
    pridectedList.append(key_list[val_list.index(i)])
    

label_1
label_2
label_2
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_1
label_5
label_5
label_1
label_2
label_5
label_5
label_5
label_3
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_1
label_5
label_5
label_1
label_1
label_3
label_5
label_3
label_1
label_5
label_5
label_5
label_5
label_5
label_1
label_5
label_2
label_5
label_1
label_5
label_1
label_5
label_5
label_5
label_2
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_3
label_5
label_5
label_1
label_5
label_5
label_5
label_1
label_1
label_5
label_5
label_5
label_5
label_5
label_2
label_5
label_5
label_5
label_1
label_1
label_5
label_5
label_5
label_1
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_5
label_2
label_5
label_2
label_5
label_5
label_5
label_5
label_5
label_1
label_5
label_4
label_1
label_1
label_5
label_5
label_1
label_2
label_2
label_1
label_1
label_1
label_1
label_5
label_5
label_5


In [50]:
pridectedList

['label_1',
 'label_2',
 'label_2',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_1',
 'label_5',
 'label_5',
 'label_1',
 'label_2',
 'label_5',
 'label_5',
 'label_5',
 'label_3',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_1',
 'label_5',
 'label_5',
 'label_1',
 'label_1',
 'label_3',
 'label_5',
 'label_3',
 'label_1',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_1',
 'label_5',
 'label_2',
 'label_5',
 'label_1',
 'label_5',
 'label_1',
 'label_5',
 'label_5',
 'label_5',
 'label_2',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_3',
 'label_5',
 'label_5',
 'label_1',
 'label_5',
 'label_5',
 'label_5',
 'label_1',
 'label_1',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_5',
 'label_2',
 'label_5',
 'label_5',
 'la